In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from utils import load_data
from constants import *


TypeError: 'type' object is not subscriptable

In [ ]:
YEARS = [2014, 2015]
reddit = {}

In [ ]:
for year in YEARS:
    reddit[year] = load_data(DATA_PATH, year=year, tokenize=False, comp="parquet", dev=False)

In [ ]:
users = {}

for year in YEARS:
    for row in 